In [1]:
import pandas as pd
import data

filepath = "../data/papers/urban-agriculture.csv"
df = pd.read_csv(filepath)
df = data.load_scopus(df)

2020-06-07 01:44:52,567 - INFO - Renaming and selecting columns ...
2020-06-07 01:44:58,356 - INFO - Formatting author names ...
2020-06-07 01:44:58,362 - INFO - Disambiguating author names ...
2020-06-07 01:44:58,432 - INFO - Removing part of titles in foreing languages ...
2020-06-07 01:44:58,440 - INFO - Building thesaurus for author and index keywords ...
2020-06-07 01:45:00,039 - INFO - Cleaning Author Keywords ...
2020-06-07 01:45:00,544 - INFO - Cleaning Index Keywords ...
2020-06-07 01:45:01,759 - INFO - Fusioning cleaned author and index keywords ...
2020-06-07 01:45:01,770 - INFO - NumExpr defaulting to 8 threads.
2020-06-07 01:45:01,784 - INFO - Extracting countries from affiliations ...
2020-06-07 01:45:07,857 - INFO - Extracting institutions from affiliations ...
2020-06-07 01:45:07,874 - INFO - Counting number of authors ...


In [2]:
import viewers

viewers.matrix(df, top_n=50)

GridspecLayout(children=(HTML(value="<h1>Matrix Explorer (Top 50 terms)</h1><hr style='height:2px;border-width…

In [ ]:
df.columns

In [ ]:
df['Author Keywords (Cleaned)']

In [ ]:
import viewers

viewers.column(df, top_n=50)

### Basic info

In [ ]:
data.coverage(df)

In [ ]:
data.app(df)

### Keywords

### FindString Dialog Box pag. 69

### Cleaning a list pag. 102

### Fuzzy match editor pag.126

### Stemming And / Stemming Or

### APPS

In [ ]:
import by_year as year
year.app(df)

In [ ]:
import by_term as term
term.app(df)

In [ ]:
import by_term_per_year as term_year
term_year.app(df)

In [ ]:
import co_occurrence as co_occurrence
co_occurrence.app(df)

In [ ]:
import correlation as correlation
correlation.app(df)

In [ ]:
import factor_analysis

factor_analysis.app(df)

In [3]:
'hola undo'.split(';')

['hola undo']

In [ ]:
s = df[['Author Keywords (Cleaned)']]
s = s['Author Keywords (Cleaned)'].map(lambda w: w.split(';') if pd.isna(w) is False else pd.NA)
s = pd.Series(s.explode().unique()).sort_values()
s[400:450]